In [ ]:
import warnings
import numpy as np
# avoid DeprecationWarning: np.find_common_type is deprecated due to pandas version (needed by other packages)
warnings.filterwarnings("ignore", category=DeprecationWarning, module="pandas.core.algorithms")

In [ ]:
import logging
from zero_shot_validation_scripts.utils import umap_on_embedding
from zero_shot_validation_scripts.dataset_preparation import load_and_preprocess_dataset

In [ ]:
#### Parameters ####

dataset_name = snakemake.wildcards.dataset

In [ ]:
result_metrics_dict = {}

#### Load data
adata = load_and_preprocess_dataset(dataset_name=dataset_name, read_count_table_path = snakemake.input.raw_read_count_table,
                                    obsm_paths={
                                        "X_embeddings": (snakemake.input.processed_dataset, "transcriptome_embeds"),
                                    })
logging.info(f"Data loaded and preprocessed. Shape: {adata.shape}")

In [ ]:
# Calculate UMAPs based on the embeddings
adata = umap_on_embedding(
    adata,
    embedding_key=f"X_embeddings",
    neighbors_key=f"neighbors_embeddings",
    umap_key=f"X_umap_on_neighbors_embeddings",
)

In [ ]:
# write the obsm  to an output file as npz
np.savez(snakemake.output.umap, neighbors=adata.obsm["X_umap_on_neighbors_embeddings"])